In [1]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess = clip.load("ViT-B/32", device=device)
model.eval()



100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 50.0MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [7]:
from datasets import load_dataset, load_from_disk  # ou load_from_disk selon ton cas
ds = load_from_disk("data/mmimdb_merged.parquet")

In [43]:
img = ds[28]["image"]          # déjà une image PIL
image = img.convert("RGB")          # au cas où
image_input = preprocess(image).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image_input)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)


In [44]:
for label, p in zip(labels, probs):
    print(f"{label}: {p:.3f}")
print(ds[24]["answer"])

Drama: 0.345
fantasy: 0.022
musical: 0.038
Comedy: 0.596
Drama


In [19]:
from datasets import load_dataset
ds2 = load_dataset("parquet", data_files="data/mmimdb_merged.parquet", split="train")

In [37]:
print(ds2[28])

{'messages': [{'role': 'user', 'content': "Given the movie poster and the corresponding plot of the movie, choose the appropriategenres from the following comma separated genres: drama, comedy, romance, thriller, crime, action, adventure,horror, documentry, mystery, sci-fi, fantasy, family, biography, war, history, music, animation, musical,western, sport, short, film-noir.\nPlot: The plot of this movie is not clear. The guy's wife informs him of her plans to divorce him at a counseling session. Husband is dumbfounded. He has several one night stands. Attends a class reunion and it appears that he wants to rekindle a relationship with an ex-girlfriend. At the end, wife approaches him about his new lifestyle and husband finally admits he now knows about her affair and that she has little room to speak when it comes to being a good role model for their daughter.\nNote that a movie can belong to more than one genres,provide all the suitable genres separated by commas. Answer:"}, {'role': 

In [38]:
img = ds2[28]["images"][0]          # déjà une image PIL
image = img.convert("RGB")          # au cas où
image_input = preprocess(image).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image_input)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)

In [39]:
labels = ["Drama",
          "fantasy",
          "musical",
          "Comedy"]

text_tokens = clip.tokenize(labels).to(device)


In [40]:
with torch.no_grad():
    # 1. features image
    image_features = model.encode_image(image_input)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)

    # 2. features texte
    text_features = model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # 3. similarités cosinus
    logits = (100.0 * image_features @ text_features.T)  # [1, nb_labels]
    probs = logits.softmax(dim=-1).cpu().numpy()[0]


In [41]:
for label, p in zip(labels, probs):
    print(f"{label}: {p:.3f}")
print(ds[24]["answer"])

Drama: 0.345
fantasy: 0.022
musical: 0.038
Comedy: 0.596
Drama


In [45]:
image.show()